# Language Knowledge (Vocabulary)
Duration: 30 minutes
Content: This section tests your knowledge of Japanese vocabulary, including kanji readings, orthography, word formation, contextually-defined expressions, paraphrases, and usage
It mainly composes following five categories:
- ``Reading Kana`` (Pronunciation Questions): Given a kanji word, choose the correct kana reading.
- `Writing Kanji` (Writing Questions): Given a word written in kana, choose the correct kanji representation.
- `Word Meaning` Selection (Vocabulary Understanding): Choose the most suitable word to fill in the sentence from four options.
- `Synonym Replacement`: Select a word that has the same or similar meaning as the underlined word.
- `Vocabulary Usage`: Assess the usage of words in actual contexts, choosing the most appropriate word usage, including some common Japanese expressions or fixed phrases.

In [1]:
import pandas as pd
import json
import os
import pickle
import re
import uuid
from typing import *
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Latex
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

load_dotenv()

True

In [2]:
# Import N3 Vocabulary
file_path = '../Vocab/n3.csv'
# Read the CSV file
data = pd.read_csv(file_path)
words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
# Display the content of the CSV file
words.head()

,expression,reading
0,近視,きんし
1,完成,かんせい
2,食品,しょくひん
3,清む,すむ
4,炒める,いためる


#### load Models

In [3]:
# azure_llm = AzureChatOpenAI(
#     azure_endpoint="https://tooldev-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview",
#     api_key=os.environ["AZURE_API_KEY"],
#     model_name="gpt-4o",
#     api_version="2025-01-01-preview",
#     temperature=0.5,
# )

In [4]:
aws_llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
     # model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    model_kwargs=dict(temperature=0.3),
    region = "us-east-2",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
)

## Question Explore

# Kanji 读假名（读音问题）

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class QuestionState(TypedDict):
    topic: str
    question: str
    improved_question: str
    final_question: str


kanji_example = """
 問題５

 (1)
日本に留学に来る前、母が持っていきなさいと言って、私の国でよく売っている粉の香辛料をくれました。私が普段あまり使わないものでしたが、役に立つかもしれないと母が言うので、荷物に入れました。最近、それが本当に役に立ちました。  
先月、①ちょっと困ったことがありました。ある留学生交流会に、国の料理を作ってきてほしいと言われたのです。私にも得意な料理はあるのですが、日本では買えない材料を使うので作れません。そのとき、私はあの香辛料を思い出したのです。  
私は、肉と卵を使ってチャーハンを作り、香辛料をかけてみました。すると、日本によくある普通のチャーハンが、私の国らしい味と香りの②特別なチャーハンになったのです。交流会でも、みんな、おいしいおいしいと言って食べてくれて、安心しました。  
あのとき、母はこういうことを予想していたのでしょうか。明日電話するので、③母に聞いてみようと思います。

---

27. ①ちょっと困ったことがありましたとあるが、「私」が困ったのはなぜか。**  
1. 母に、普段あまり使わない香辛料を持っていくように言われたから  
2. 得意な料理がないのに、国の料理を作ってきてほしいと言われたから  
3. 国の料理を作ってきてほしいと言われたが、日本では得意な料理が作れないから  
4. チャーハンが作れないのに、チャーハンを作ってきてほしいと言われたから  

28. ②特別なチャーハンとは、どのようなチャーハンか。**  
1. 日本で売っている材料でチャーハンを作って、国の香辛料をかけたもの  
2. 日本で売っている材料でチャーハンを作って、日本の香辛料をかけたもの  
3. 国から持ってきた材料でチャーハンを作って、国の香辛料をかけたもの  
4. 国から持ってきた材料でチャーハンを作って、日本の香辛料をかけたもの  

29. ③母に聞いてみようと思いますとあるが、「私」はどのようなことを聞くと考えられるか。**  
1. 国の香辛料がどうして日本で役に立つと思ったのか  
2. どんな料理を作るときに国の香辛料を使えばいいのか  
3. 「私」が日本に留学することを予想していたかどうか

 問題５

 (2)
服や靴を作っている、ある海外のファッションの会社が始めた活動がある。まず、漁師たち（魚をとって生活している人たち）に組んで、魚をとるときに網にとれるごみを港に持ち帰ってもらう。そして、会社がそのごみを回収・分割し、その中のプラスチックを縫紐に変え、服や靴にして売るというリサイクル活動である。  
以前、漁師たちはごみがとれても海に戻していた。港に持ち帰ると捨てるのにお金がかかるからだ。この活動は、漁師にとっても、自分のお金を使わずに海をきれいにできる良さがある。  
これらの服や靴は最近日本でも売られ始めた。デザインも悪くない。消費者の意識が変化している今、日本でもきっと受け入れられるだろう。

---

30. 「海外のファッションの会社」がしていることとして、合っているのはどれか。**  
1. 漁師たちと一緒に、海にごみをとりに行っている  
2. 漁師たちから港でごみを受け取って、他に持ち帰っている  
3. 漁師たちがプラスチックごみから服や靴を作るのを助けて、それを売っている  
4. 漁師たちが海でとったプラスチックごみを利用して、服や靴を作っている  

31. 漁師たちは、なぜ「海外のファッションの会社」が始めた活動に参加するのか。**  
1. ほかの漁師たちとの協力関係ができるから  
2. 自分たちのお金をかけずに、海のごみを減らすことができるから  
3. 魚をとるためにかかっていたお金を減らすことができるから  
4. 自分たちが少しお金を出すだけで、海をきれいにしてもらえるから  

32. この文章を書いた人は、日本で売られ始めた「海外のファッションの会社」の服や靴について、どのように考えているか。**  
1. 海外のファッションに関心を持つ人が増えているので、売れるだろう  
2. 環境問題に関心を持つ人が増えているし、デザインも悪くないので、売れるだろう  
3. デザインの良さで製品を選ぶ人が増えているので、値段が高くても売れるだろう
"""

print(words)

# Nodes
def generate_question(state: QuestionState):
    """First LLM call to generate initial question"""

    msg = aws_llm.invoke(f"You are a japanese teacher. Your job is Read 2 articles with 10-11 lines, with a word count of around 400 words, and answer approximately 3 questions. Only refer to the format, not the content."
            f"The vocabulary should be restricted to N3 level, you can refer to the vocabulary in the word list, choosing random words for the questions"
            f"please refer the question examples following the formal exam paper"
            f"append the correct answer and explanation of main challenges and why teacher asks this question to candidate in chinese at each question"
            f"Finally, beautify markdown format"
            f"topic list: {state['topic']}"
            f"word list: {words}"
            f"formal exam paper: {kanji_example}")
    
    return {"question": msg.content}


# Build workflow
kanji_workflow = StateGraph(QuestionState)

# Add nodes
kanji_workflow.add_node("generate_question", generate_question)

# Add edges to connect nodes
kanji_workflow.add_edge(START, "generate_question")
kanji_workflow.add_edge("generate_question", END)

# Compile
kanji_graph = kanji_workflow.compile()


     expression reading
0            近視     きんし
1            完成    かんせい
2            食品   しょくひん
3            清む      すむ
4           炒める    いためる
...         ...     ...
2134         地味      じみ
2135       なぜなら    なぜなら
2136         不安     ふあん
2137         公演    こうえん
2138         性質    せいしつ

[2139 rows x 2 columns]


In [6]:
# Show workflow
#display(Image(kanji_graph.get_graph().draw_png()))

In [7]:
# Invoke
kanji = kanji_graph.invoke({"topic": "店で価格を尋ねる | レストランで食べ物を注文する | 交通手段について話す| 天気の状況について話す | 趣味について話す | 店で商品の特徴や価格を尋ねる | スーパーマーケットで買い物をする | レストランで食べ物や飲み物を注文する | カフェでテイクアウトを頼む | コンビニで支払い方法について質問する |  ネットショッピングで商品を選ぶ"})
display(Markdown(kanji["question"]))

Error raised by bedrock service
Traceback (most recent call last):
  File "c:\Users\HW-BK003\Desktop\jlpt_simulator\.venv\Lib\site-packages\langchain_aws\llms\bedrock.py", line 956, in _prepare_input_and_invoke
    response = self.client.invoke_model(**request_options)
  File "c:\Users\HW-BK003\Desktop\jlpt_simulator\.venv\Lib\site-packages\botocore\client.py", line 570, in _api_call
    return self._make_api_call(operation_name, kwargs)
           ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HW-BK003\Desktop\jlpt_simulator\.venv\Lib\site-packages\botocore\context.py", line 124, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\HW-BK003\Desktop\jlpt_simulator\.venv\Lib\site-packages\botocore\client.py", line 1031, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Access to Anthropic models is not allowed from unsupp

ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Access to Anthropic models is not allowed from unsupported countries, regions, or territories. Please refer to https://www.anthropic.com/supported-countries for more information on the countries and regions Anthropic currently supports.